<a href="https://colab.research.google.com/github/uozumi-n/kaigisitu_iio/blob/main/%E4%BC%9A%E8%AD%B0%E5%AE%A4%E3%82%B7%E3%83%BC%E3%83%88%E4%B8%80%E6%AC%A1%E9%9B%86%E8%A8%88%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ①インスタベース利用データ集計プログラム（最終確定版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone

def perform_aggregation(df):
    """インスタベースのデータフレームを受け取り、集計を実行"""
    # 1. データのクリーニング（集計対象：予約金額 (税込)）
    target_sales_col = '予約金額 (税込)'

    if target_sales_col in df.columns:
        def clean_currency(x):
            if pd.isna(x): return 0
            s = re.sub(r'[^0-9.-]', '', str(x))
            return pd.to_numeric(s, errors='coerce')
        df['売上'] = df[target_sales_col].apply(clean_currency).fillna(0)
    else:
        raise ValueError(f"カラム '{target_sales_col}' が見つかりません。")

    # 【重要】日付変換を強化（format='mixed' を使用）
    df['開始'] = pd.to_datetime(df['利用開始日時'], errors='coerce', format='mixed')
    df['終了'] = pd.to_datetime(df['利用終了日時'], errors='coerce', format='mixed')
    df['利用時間_td'] = df['終了'] - df['開始']
    df['集計日'] = df['開始'].dt.normalize()

    # 無効な行を除去
    df = df.dropna(subset=['集計日', '利用時間_td'])

    # 2. グルーピング集計
    group_keys = ['集計日', '施設名', 'スペース名']
    summary = df.groupby(group_keys, dropna=False).agg(
        件数=('予約ID', 'size'),
        利用時間_合計=('利用時間_td', 'sum'),
        売上_合計=('売上', 'sum')
    ).reset_index()

    # 3. 期間フィルタリング（昨日以前）
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    # 4. フォーマット変換
    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = summary['利用時間_合計'].apply(timedelta_to_hhmm)
    summary['売上'] = summary['売上_合計'].round().astype(int)
    summary['設備名'] = summary['スペース名']
    summary['プラン名'] = summary['スペース名']

    return summary[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上']]

def load_csv(uploaded_file):
    filename = next(iter(uploaded_file))
    try:
        return pd.read_csv(io.BytesIO(uploaded_file[filename]), encoding='utf-8-sig')
    except:
        return pd.read_csv(io.BytesIO(uploaded_file[filename]), encoding='cp932')

try:
    print("【手順1】タグ設置リストをアップロードしてください。")
    up_tag = files.upload()
    df_tag_master = load_csv(up_tag)
    df_mapping = df_tag_master[df_tag_master['媒体名'] == 'instabase'].copy()
    df_mapping = df_mapping[['媒体側施設名', '媒体側設備名', '統一施設名', '統一設備名']].drop_duplicates()
    df_mapping = df_mapping.rename(columns={'媒体側施設名': 'f_key', '媒体側設備名': 'e_key', '統一施設名': '統一店名'})

    print("\n【手順2】インスタベースの利用データをアップロードしてください。")
    up_data = files.upload()
    df_input = load_csv(up_data)

    res_df = perform_aggregation(df_input)

    # 照合
    res_df['f_tmp'] = res_df['施設名'].astype(str).str.strip()
    res_df['e_tmp'] = res_df['設備名'].astype(str).str.strip()
    df_mapping['f_tmp'] = df_mapping['f_key'].astype(str).str.strip()
    df_mapping['e_tmp'] = df_mapping['e_key'].astype(str).str.strip()

    merged_df = pd.merge(res_df, df_mapping[['f_tmp', 'e_tmp', '統一店名', '統一設備名']], on=['f_tmp', 'e_tmp'], how='left')
    unmatched = merged_df[merged_df['統一店名'].isna()][['施設名', '設備名']].drop_duplicates()

    merged_df['統一店名'] = merged_df['統一店名'].fillna('')
    merged_df['統一設備名'] = merged_df['統一設備名'].fillna('')

    out_name = 'インスタベース集計結果_統合版.csv'
    merged_df.drop(columns=['f_tmp', 'e_tmp']).to_csv(out_name, index=False, encoding='utf-8-sig')
    files.download(out_name)
    if not unmatched.empty:
        unmatched.to_csv('紐付け失敗リスト_インスタ.csv', index=False, encoding='utf-8-sig')
        files.download('紐付け失敗リスト_インスタ.csv')
    print("\n✅ 完了しました。")
except Exception as e:
    print(f"\n❌ エラー: {e}")

【手順1】タグ設置リストをアップロードしてください。


Saving 会議室シート_改 - タグ設置リスト (4).csv to 会議室シート_改 - タグ設置リスト (4).csv

【手順2】インスタベースの利用データをアップロードしてください。


Saving 他媒体データ置き場 - instabase のコピー (3).csv to 他媒体データ置き場 - instabase のコピー (3).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 完了しました。


In [ ]:
#@title ②スペースマーケット売上明細集計プログラム（最終確定版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone

def perform_aggregation(df):
    """スペースマーケットのデータフレームを受け取り、集計を実行"""
    # 1. データのクリーニング（集計対象：成約金額）
    target_sales_col = '成約金額'

    if target_sales_col in df.columns:
        def clean_currency(x):
            if pd.isna(x): return 0
            s = re.sub(r'[^0-9.-]', '', str(x))
            return pd.to_numeric(s, errors='coerce')
        df['売上'] = df[target_sales_col].apply(clean_currency).fillna(0)
    else:
        raise ValueError(f"カラム '{target_sales_col}' が見つかりません。")

    # 日付変換を強化（format='mixed'）
    df['集計日'] = pd.to_datetime(df['実施日'], errors='coerce', format='mixed')

    if '利用開始時間' in df.columns and '利用終了時間' in df.columns:
        # 時間の計算も安全に行う
        df['利用時間_td'] = pd.to_datetime(df['利用終了時間'], format='mixed') - pd.to_datetime(df['利用開始時間'], format='mixed')
    else:
        df['利用時間_td'] = pd.Timedelta(0)

    df = df.dropna(subset=['集計日'])

    # 2. グルーピング集計
    group_keys = ['集計日', '施設名', 'プラン名', 'スペース名']
    summary = df.groupby(group_keys, dropna=False).agg(
        件数=('予約ID', 'size'),
        利用時間_合計=('利用時間_td', 'sum'),
        売上_合計=('売上', 'sum')
    ).reset_index()

    # 3. 期間フィルタリング（昨日以前）
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    # 4. フォーマット変換
    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = summary['利用時間_合計'].apply(timedelta_to_hhmm)
    summary['売上'] = summary['売上_合計'].round().astype(int)

    # 【重要】プラン名とスペース名の入れ替え
    # 出力「プラン名」＝元の「スペース名」
    # 出力「設備名」＝元の「プラン名」（マスタの「媒体側設備名」と照合するため）
    summary['orig_plan'] = summary['プラン名']
    orig_p = summary['プラン名'].copy()
    orig_s = summary['スペース名'].copy()
    summary['プラン名'] = orig_s
    summary['設備名'] = orig_p

    return summary[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上', 'orig_plan']]

def load_csv_with_encoding(uploaded_file_dict):
    """文字コードを自動判定して読み込むヘルパー関数"""
    filename = next(iter(uploaded_file_dict))
    content = uploaded_file_dict[filename]
    try:
        # まずは UTF-8 で試行
        return pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
    except UnicodeDecodeError:
        # 失敗したら Shift-JIS で試行
        return pd.read_csv(io.BytesIO(content), encoding='cp932')

# --- メイン実行 ---
try:
    # 1. マスタの読み込み
    print("【手順1】タグ設置リストをアップロードしてください。")
    up_tag = files.upload()
    df_tag_master = load_csv_with_encoding(up_tag)

    # マッピング準備
    df_mapping = df_tag_master[df_tag_master['媒体名'] == 'spacemarket'].copy()
    df_mapping = df_mapping[['媒体側施設名', '媒体側設備名', '統一施設名', '統一設備名']].drop_duplicates()
    df_mapping = df_mapping.rename(columns={'媒体側施設名': 'f_key', '媒体側設備名': 'e_key', '統一施設名': '統一店名'})

    # 2. 売上明細の読み込み
    print("\n【手順2】スペースマーケットの売上明細をアップロードしてください。")
    up_data = files.upload()
    df_input = load_csv_with_encoding(up_data)

    # 3. 集計実行
    res_df = perform_aggregation(df_input)

    # 4. マスタ照合（一時キー作成）
    res_df['f_tmp'] = res_df['施設名'].astype(str).str.strip()
    res_df['p_tmp'] = res_df['orig_plan'].astype(str).str.strip()
    df_mapping['f_tmp'] = df_mapping['f_key'].astype(str).str.strip()
    df_mapping['p_tmp'] = df_mapping['e_key'].astype(str).str.strip()

    # 結合
    merged_df = pd.merge(res_df, df_mapping[['f_tmp', 'p_tmp', '統一店名', '統一設備名']], on=['f_tmp', 'p_tmp'], how='left')

    # 判定
    unmatched = merged_df[merged_df['統一店名'].isna()][['施設名', 'orig_plan']].drop_duplicates()

    # 空欄補完
    merged_df['統一店名'] = merged_df['統一店名'].fillna('')
    merged_df['統一設備名'] = merged_df['統一設備名'].fillna('')

    # 不要な一時カラムを削除して列順確定
    final_output = merged_df[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上', '統一店名', '統一設備名']]

    # 5. 出力
    out_name = 'スペースマーケット集計結果_統合版.csv'
    final_output.to_csv(out_name, index=False, encoding='utf-8-sig')
    files.download(out_name)

    if not unmatched.empty:
        unmatched.to_csv('紐付け失敗リスト_スペマ.csv', index=False, encoding='utf-8-sig')
        files.download('紐付け失敗リスト_スペマ.csv')
        print(f"⚠️ 照合できなかった項目があります。失敗リストを確認してください。")

    print("\n✅ 完了しました。")
except Exception as e:
    print(f"\n❌ エラー: {e}")

【手順1】タグ設置リストをアップロードしてください。


Saving 会議室シート_改 - タグ設置リスト.csv to 会議室シート_改 - タグ設置リスト (8).csv

【手順2】スペースマーケットの売上明細をアップロードしてください。


Saving 他媒体データ置き場 - spacemarket のコピー.csv to 他媒体データ置き場 - spacemarket のコピー (3).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⚠️ 照合できなかった項目があります。失敗リストを確認してください。

✅ 完了しました。


In [ ]:
#@title ③スペイシー利用データ集計プログラム（最終確定版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone

def perform_aggregation(df):
    """スペイシーのデータフレームを受け取り、集計を実行"""
    # 1. データのクリーニング（集計対象：差引合計売上金額（税込））
    target_sales_col = '差引合計売上金額（税込）'

    if target_sales_col in df.columns:
        def clean_currency(x):
            if pd.isna(x): return 0
            if isinstance(x, (int, float)): return x
            s = re.sub(r'[^0-9.-]', '', str(x))
            return pd.to_numeric(s, errors='coerce')
        df['売上'] = df[target_sales_col].apply(clean_currency).fillna(0)
    else:
        raise ValueError(f"カラム '{target_sales_col}' が見つかりません。")

    # ステータスフィルタリング
    if '予約ステータス' in df.columns:
        df = df[df['予約ステータス'] == '予約完了'].copy()

    # 日付変換を強化（format='mixed' を使用し、混合形式に対応）
    df['開始'] = pd.to_datetime(df['利用開始日時'], errors='coerce', format='mixed')
    df['終了'] = pd.to_datetime(df['利用終了日時'], errors='coerce', format='mixed')
    df['利用時間_td'] = df['終了'] - df['開始']
    df['集計日'] = df['開始'].dt.normalize()

    # 無効な行を除去
    df = df.dropna(subset=['集計日', '利用時間_td'])

    # 2. グルーピング集計
    group_keys = ['集計日', 'スペース名']
    summary = df.groupby(group_keys, dropna=False).agg(
        件数=('予約ID', 'size'),
        利用時間_合計=('利用時間_td', 'sum'),
        売上_合計=('売上', 'sum')
    ).reset_index()

    # 3. 期間フィルタリング（昨日以前）
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    # 4. フォーマット変換
    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = summary['利用時間_合計'].apply(timedelta_to_hhmm)
    summary['売上'] = summary['売上_合計'].round().astype(int)

    # スペイシーの仕様に合わせてマッピング
    summary['施設名'] = summary['スペース名']
    summary['プラン名'] = summary['スペース名']
    summary['設備名'] = summary['スペース名']

    return summary[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上']]

def load_csv_with_encoding(uploaded_file_dict):
    """文字コードを自動判定して読み込むヘルパー関数"""
    filename = next(iter(uploaded_file_dict))
    content = uploaded_file_dict[filename]
    try:
        # まずは UTF-8 (BOM付き含む) で試行
        return pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
    except UnicodeDecodeError:
        # 失敗したら Shift-JIS (cp932) で試行
        return pd.read_csv(io.BytesIO(content), encoding='cp932')

# --- メイン実行 ---
try:
    # 1. マスタの読み込み
    print("【手順1】タグ設置リストをアップロードしてください。")
    up_tag = files.upload()
    df_tag_master = load_csv_with_encoding(up_tag)

    # マッピング準備（spacee）
    df_mapping = df_tag_master[df_tag_master['媒体名'] == 'spacee'].copy()
    df_mapping = df_mapping[['媒体側施設名', '媒体側設備名', '統一施設名', '統一設備名']].drop_duplicates()
    df_mapping = df_mapping.rename(columns={'媒体側施設名': 'f_key', '媒体側設備名': 'e_key', '統一施設名': '統一店名'})

    # 2. 利用データの読み込み
    print("\n【手順2】スペイシーの利用データをアップロードしてください。")
    up_data = files.upload()
    df_input = load_csv_with_encoding(up_data)

    # 3. 集計実行
    res_df = perform_aggregation(df_input)

    # 4. マスタ照合（設備名 ＝ スペイシーのスペース名 で照合）
    res_df['e_tmp'] = res_df['設備名'].astype(str).str.strip()
    df_mapping['e_tmp'] = df_mapping['e_key'].astype(str).str.strip()

    # 結合
    merged_df = pd.merge(res_df, df_mapping[['e_tmp', '統一店名', '統一設備名']], on=['e_tmp'], how='left')

    # 紐付け失敗チェック
    unmatched = merged_df[merged_df['統一店名'].isna()][['設備名']].drop_duplicates()

    # 空欄補完
    merged_df['統一店名'] = merged_df['統一店名'].fillna('')
    merged_df['統一設備名'] = merged_df['統一設備名'].fillna('')

    # 列順の確定
    final_output = merged_df[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上', '統一店名', '統一設備名']]

    # 5. 出力
    out_name = 'スペイシー集計結果_統合版.csv'
    final_output.to_csv(out_name, index=False, encoding='utf-8-sig')
    files.download(out_name)

    if not unmatched.empty:
        unmatched.to_csv('紐付け失敗リスト_スペイシー.csv', index=False, encoding='utf-8-sig')
        files.download('紐付け失敗リスト_スペイシー.csv')
        print(f"⚠️ マスタにないスペースがありました。失敗リストを確認してください。")

    print("\n✅ 完了しました。")
except Exception as e:
    print(f"\n❌ エラー: {e}")

【手順1】タグ設置リストをアップロードしてください。


Saving 会議室シート_改 - タグ設置リスト.csv to 会議室シート_改 - タグ設置リスト (10).csv

【手順2】スペイシーの利用データをアップロードしてください。


Saving 他媒体データ置き場 - spacee のコピー.csv to 他媒体データ置き場 - spacee のコピー (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⚠️ マスタにないスペースがありました。失敗リストを確認してください。

✅ 完了しました。


# **[いいアプリ元データの入手はこちらから](https://e-office.metabaseapp.com/public/dashboard/12bc87a0-0ae5-4302-9957-5b208cbd54ca)**<br>元データをダウンロードしたら次のステップに移ってください。

In [ ]:
#@title ④いいアプリ利用データ集計プログラム（最終確定版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone

def perform_aggregation(df_main, df_ext):
    """通常分と延長分を合算し、昨日以前のデータを集計する関数"""
    MIN_SALES_THRESHOLD = 1

    # 強力な数値変換関数
    def clean_currency(x):
        if pd.isna(x): return 0
        if isinstance(x, (int, float)): return x
        s = re.sub(r'[^0-9.-]', '', str(x).replace('￥', '').replace('¥', '').replace(',', ''))
        return pd.to_numeric(s, errors='coerce')

    # 通常・延長それぞれの売上を数値化
    for df in [df_main, df_ext]:
        if '売上' in df.columns:
            df['売上'] = df['売上'].apply(clean_currency).fillna(0)

    # --- 1. 通常データの処理 ---
    df_main = df_main[df_main['売上'] >= MIN_SALES_THRESHOLD].copy()
    # 日付変換（format='mixed'で混合形式に対応）
    df_main['集計日'] = pd.to_datetime(df_main['売上確定日'], errors='coerce', format='mixed')

    # 時間計算用のクレンジング
    def clean_time_str(s):
        if pd.isna(s): return s
        return str(s).replace(', ', ' ').strip()

    start_t = pd.to_datetime(df_main['開始'].apply(clean_time_str), errors='coerce', format='mixed')
    end_act = pd.to_datetime(df_main['実際の終了時間'].apply(clean_time_str), errors='coerce', format='mixed')
    end_sch = pd.to_datetime(df_main['終了'].apply(clean_time_str), errors='coerce', format='mixed')

    df_main['利用時間_td'] = end_act.fillna(end_sch) - start_t
    df_main = df_main.dropna(subset=['集計日', '利用時間_td'])

    # --- 2. 延長データの処理 ---
    df_ext = df_ext[df_ext['売上'] >= MIN_SALES_THRESHOLD].copy()
    df_ext['集計日'] = pd.to_datetime(df_ext['売上確定日'], errors='coerce', format='mixed')
    df_ext['延長時間_td'] = pd.to_timedelta(df_ext['延長した分数'], unit='m', errors='coerce')
    df_ext = df_ext.dropna(subset=['集計日', '延長時間_td'])

    # --- 3. 集計と合算 ---
    group_keys = ['集計日', '施設名', 'プラン名', '設備名']

    summary_main = df_main.groupby(group_keys, dropna=False).agg(
        件数=('施設名', 'size'),
        通常時間_td=('利用時間_td', 'sum'),
        通常売上=('売上', 'sum')
    ).reset_index()

    summary_ext = df_ext.groupby(group_keys, dropna=False).agg(
        延長時間_td=('延長時間_td', 'sum'),
        延長売上=('売上', 'sum')
    ).reset_index()

    # 外部結合
    merged = pd.merge(summary_main, summary_ext, on=group_keys, how='outer')

    # 欠損値を補完
    merged['件数'] = merged['件数'].fillna(0).astype(int)
    merged['売上'] = (merged['通常売上'].fillna(0) + merged['延長売上'].fillna(0)).astype(int)
    total_time_td = merged['通常時間_td'].fillna(pd.Timedelta(0)) + merged['延長時間_td'].fillna(pd.Timedelta(0))

    # --- 4. 期間フィルタ（昨日以前） ---
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    merged = merged[merged['集計日'].dt.date <= yesterday_in_japan].copy()

    # --- 5. フォーマット変換 ---
    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    merged['月次タグ'] = merged['集計日'].dt.strftime('%Y年%m月')
    merged['日付'] = merged['集計日'].dt.strftime('%Y-%m-%d')
    merged['利用時間'] = total_time_td.apply(timedelta_to_hhmm)

    return merged[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上']]

def load_csv_with_encoding(uploaded_file_dict):
    """文字コードを自動判定して読み込む"""
    filename = next(iter(uploaded_file_dict))
    content = uploaded_file_dict[filename]
    try:
        return pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
    except UnicodeDecodeError:
        return pd.read_csv(io.BytesIO(content), encoding='cp932')

# --- メイン実行部分 ---
try:
    print("【手順1】タグ設置リストをアップロードしてください。")
    up_tag = files.upload()
    df_tag_master = load_csv_with_encoding(up_tag)
    df_mapping = df_tag_master[df_tag_master['媒体名'] == 'いいアプリ'].copy()
    df_mapping = df_mapping[['媒体側施設名', '媒体側設備名', '統一施設名', '統一設備名']].drop_duplicates()
    df_mapping = df_mapping.rename(columns={'媒体側施設名': 'f_key', '媒体側設備名': 'e_key', '統一施設名': '統一店名'})

    print("\n【手順2】「①設備予約（延長以外）.csv」をアップロードしてください。")
    up_main = files.upload()
    df_main_in = load_csv_with_encoding(up_main)

    print("\n【手順3】「②設備予約（延長のみ）.csv」をアップロードしてください。")
    up_ext = files.upload()
    df_ext_in = load_csv_with_encoding(up_ext)

    print("\n集計と照合を開始します...")
    res_df = perform_aggregation(df_main_in, df_ext_in)

    # 照合用キー作成
    res_df['f_tmp'] = res_df['施設名'].astype(str).str.strip()
    res_df['e_tmp'] = res_df['設備名'].astype(str).str.strip()
    df_mapping['f_tmp'] = df_mapping['f_key'].astype(str).str.strip()
    df_mapping['e_tmp'] = df_mapping['e_key'].astype(str).str.strip()

    # 結合
    merged_df = pd.merge(res_df, df_mapping[['f_tmp', 'e_tmp', '統一店名', '統一設備名']], on=['f_tmp', 'e_tmp'], how='left')
    unmatched = merged_df[merged_df['統一店名'].isna()][['施設名', '設備名']].drop_duplicates()

    merged_df['統一店名'] = merged_df['統一店名'].fillna('')
    merged_df['統一設備名'] = merged_df['統一設備名'].fillna('')

    out_name = 'いいアプリ利用実績_集計結果_統合版.csv'
    merged_df[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上', '統一店名', '統一設備名']].to_csv(out_name, index=False, encoding='utf-8-sig')
    files.download(out_name)

    if not unmatched.empty:
        unmatched.to_csv('紐付け失敗リスト_いいアプリ.csv', index=False, encoding='utf-8-sig')
        files.download('紐付け失敗リスト_いいアプリ.csv')
    print("\n✅ 完了しました。")
except Exception as e:
    print(f"\n❌ エラー: {e}")

【手順1】タグ設置リストをアップロードしてください。


Saving 会議室シート_改 - タグ設置リスト (3).csv to 会議室シート_改 - タグ設置リスト (3).csv

【手順2】「①設備予約（延長以外）.csv」をアップロードしてください。


KeyboardInterrupt: 

In [ ]:
#@title ⑤全媒体データ統合ツール（日別集計・不明データ除外版）

# ===================================================================
#               全媒体データ統合プログラム（日別版）
# ===================================================================

import pandas as pd
from google.colab import files
import io
import unicodedata
import re

def merge_media_data():
    print("各媒体の集計済みCSVファイルをすべて選択してアップロードしてください。")
    uploaded = files.upload()

    if not uploaded:
        print("ファイルが選択されませんでした。")
        return

    all_dfs = []

    # 媒体判定用のキーワードマップ
    media_map = {
        'インスタベース': 'インスタベース',
        'スペースマーケット': 'スペースマーケット',
        'スペイシー': 'スペイシー',
        'いいアプリ': 'いいアプリ',
        '会議室': 'いいアプリ'
    }

    # 時間（HH:MM）を分に変換するヘルパー関数
    def time_to_minutes(s):
        try:
            if pd.isna(s) or s == "": return 0
            parts = str(s).split(':')
            if len(parts) == 2:
                return int(parts[0]) * 60 + int(parts[1])
            return 0
        except:
            return 0

    # 分を時間（HH:MM）に変換するヘルパー関数
    def minutes_to_time(m):
        m = int(m)
        return f"{m // 60}:{m % 60:02}"

    for filename, content in uploaded.items():
        filename_norm = unicodedata.normalize('NFC', filename)

        if '失敗' in filename_norm or 'エラー' in filename_norm:
            continue

        media_name = "不明"
        for key, val in media_map.items():
            if key in filename_norm:
                media_name = val
                break

        print(f"-> 「{filename}」を読み込み中... [判定媒体: {media_name}]")

        try:
            df = pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
        except:
            df = pd.read_csv(io.BytesIO(content), encoding='cp932')

        df['媒体名'] = media_name

        # カラム名の調整
        if '統一店名' not in df.columns and '統一施設名' in df.columns:
            df = df.rename(columns={'統一施設名': '統一店名'})

        # 必須データのクリーニング
        # 不明なもの、空欄のものを除外
        df['統一店名'] = df['統一店名'].astype(str).str.strip()
        df['統一設備名'] = df['統一設備名'].astype(str).str.strip()

        df = df[
            (df['媒体名'] != "不明") &
            (df['統一店名'] != "") & (df['統一店名'] != "nan") &
            (df['統一設備名'] != "") & (df['統一設備名'] != "nan")
        ].copy()

        if not df.empty:
            # 時間を数値化（集計用）
            df['利用分'] = df['利用時間'].apply(time_to_minutes)
            all_dfs.append(df)

    if not all_dfs:
        print("統合可能な有効データが見つかりませんでした。")
        return

    # 全データを結合
    combined_df = pd.concat(all_dfs, ignore_index=True)

    # --- 日別集計処理 ---
    # 月次タグ、日付、媒体名、統一店名、統一設備名でグループ化して数値を合計
    group_cols = ['月次タグ', '日付', '媒体名', '統一店名', '統一設備名']

    # 件数、売上、利用分（分単位）を合計
    summary_df = combined_df.groupby(group_cols, as_index=False).agg({
        '件数': 'sum',
        '売上': 'sum',
        '利用分': 'sum'
    })

    # 分単位の合計を HH:MM 形式に戻す
    summary_df['利用時間'] = summary_df['利用分'].apply(minutes_to_time)

    # 不要な「利用分」列を削除し、指定のカラム順に整列
    final_cols = ['月次タグ', '日付', '媒体名', '統一店名', '統一設備名', '件数', '利用時間', '売上']
    summary_df = summary_df[final_cols]

    # ソート（月次タグ、日付、店名）
    summary_df = summary_df.sort_values(['月次タグ', '日付', '媒体名', '統一店名'])

    # CSV出力
    output_filename = '全媒体日別統合データ_最終集計.csv'
    summary_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    files.download(output_filename)

    print(f"\n✅ 日別統合が完了しました！")
    print(f"「{output_filename}」をダウンロードしました。")
    print("-" * 30)
    print("【媒体別の日別データ数（行数）】")
    print(summary_df['媒体名'].value_counts())

# 実行
try:
    merge_media_data()
except Exception as e:
    print(f"エラーが発生しました: {e}")


各媒体の集計済みCSVファイルをすべて選択してアップロードしてください。


Saving 会議室シート_改 - いいアプリ (1).csv to 会議室シート_改 - いいアプリ (1).csv
Saving 会議室シート_改 - スペイシー (1).csv to 会議室シート_改 - スペイシー (1).csv
Saving 会議室シート_改 - スペースマーケット (1).csv to 会議室シート_改 - スペースマーケット (1).csv
Saving 会議室シート_改 - インスタベース (1).csv to 会議室シート_改 - インスタベース (1).csv
-> 「会議室シート_改 - いいアプリ (1).csv」を読み込み中... [判定媒体: いいアプリ]
-> 「会議室シート_改 - スペイシー (1).csv」を読み込み中... [判定媒体: スペイシー]
-> 「会議室シート_改 - スペースマーケット (1).csv」を読み込み中... [判定媒体: スペースマーケット]
-> 「会議室シート_改 - インスタベース (1).csv」を読み込み中... [判定媒体: インスタベース]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 月次統合が完了しました！
「全媒体月次統合データ_最終集計.csv」をダウンロードしました。
------------------------------
【媒体別の月次データ数（行数）】
媒体名
インスタベース      1341
いいアプリ        1276
スペースマーケット    1163
スペイシー         710
Name: count, dtype: int64


# **ここから下は龍﨑さん用のレポート作成ツール**

[売上と経費シート](https://e-office.metabaseapp.com/public/dashboard/dcf06adb-c45a-499b-bb04-fa5e933ec826)から売上と経費ぞれぞれCSVデータを入手。期間絞って更新したい月分だけにしたほうが容量軽くて楽です。

In [ ]:
# @title ⑥いいアプリ側の他売上を集計（日別対応版）

import pandas as pd
import numpy as np
import os
import io
from datetime import date
from google.colab import files

# --- 設定項目 ---
OUTPUT_FILE_NAME = 'daily_summary_final.csv'

def clean_and_convert_to_numeric(series):
    """金額の列をクリーンにして数値型に変換する関数"""
    cleaned_series = series.astype(str).str.replace(r'[^\d.]', '', regex=True)
    cleaned_series = cleaned_series.replace('', '0')
    return pd.to_numeric(cleaned_series, errors='coerce').fillna(0)

def process_sales(df, cutoff_date):
    """売上データ(DataFrame)を日別・店舗別に集計する関数"""
    df.columns = df.columns.str.strip()
    for col in ['利用プラン', '項目', '対象店舗']:
        if col in df.columns:
            df[col] = df[col].fillna('').astype(str).str.strip()

    df['発生日'] = pd.to_datetime(df['発生日'], errors='coerce')
    df.dropna(subset=['発生日'], inplace=True)
    df = df[df['発生日'].dt.date <= cutoff_date]

    if df.empty:
        return pd.DataFrame()

    # 日別・月次タグの両方を保持
    df['日付'] = df['発生日'].dt.strftime('%Y-%m-%d')
    df['月次タグ'] = df['発生日'].dt.strftime('%Y年%m月')
    df['売上'] = clean_and_convert_to_numeric(df['売上'])

    conditions = [
        (df['利用プラン'].isin(['ドロップイン利用', '法人ドロップイン利用'])),
        (df['利用プラン'] == '店舗サブスクリプション/月額契約'),
        (df['利用プラン'].isin(['設備予約', '設備予約_早上がり', '設備予約キャンセル'])),
        ((df['利用プラン'] == 'その他') & (df['項目'] == '外部媒体売上')),
    ]
    choices = ['ドロップイン売上', '店舗プラン売上', '設備予約売上', '外部媒体売上']
    df['カテゴリ'] = np.select(conditions, choices, default='その他売上')

    # 日別でグループ化
    summary = df.groupby(['月次タグ', '日付', '対象店舗', 'カテゴリ'])['売上'].sum().reset_index()
    pivot_summary = summary.pivot_table(
        index=['月次タグ', '日付', '対象店舗'], columns='カテゴリ', values='売上', fill_value=0
    ).reset_index()

    pivot_summary.rename(columns={'対象店舗': '店名'}, inplace=True)
    sales_columns = ['ドロップイン売上', '店舗プラン売上', '設備予約売上', '外部媒体売上', 'その他売上']
    for col in sales_columns:
        if col not in pivot_summary.columns:
            pivot_summary[col] = 0

    pivot_summary['売上合計'] = pivot_summary[sales_columns].sum(axis=1)
    return pivot_summary

def process_expenses(df, cutoff_date):
    """経費データ(DataFrame)を日別・店舗別に集計する関数"""
    df.columns = df.columns.str.strip()
    for col in ['Description', '施設名']:
        if col in df.columns:
            df[col] = df[col].fillna('').astype(str).str.strip()

    df['Sales Date'] = pd.to_datetime(df['Sales Date'], errors='coerce')
    df.dropna(subset=['Sales Date'], inplace=True)
    df = df[df['Sales Date'].dt.date <= cutoff_date]

    if df.empty:
        return pd.DataFrame()

    # 日別・月次タグの両方を保持
    df['日付'] = df['Sales Date'].dt.strftime('%Y-%m-%d')
    df['月次タグ'] = df['Sales Date'].dt.strftime('%Y年%m月')
    df['Amount'] = clean_and_convert_to_numeric(df['Amount'])

    conditions = [
        (df['Description'].str.contains('賃料|共益費')),
        (df['Description'].str.contains('電気')),
        (df['Description'].str.contains('水道')),
        (df['Description'].str.contains('外部媒体')),
    ]
    choices = ['賃料', '電気', '水道', '外部媒体手数料']
    df['カテゴリ'] = np.select(conditions, choices, default='その他経費')

    # 日別でグループ化
    summary = df.groupby(['月次タグ', '日付', '施設名', 'カテゴリ'])['Amount'].sum().reset_index()
    pivot_summary = summary.pivot_table(
        index=['月次タグ', '日付', '施設名'], columns='カテゴリ', values='Amount', fill_value=0
    ).reset_index()

    pivot_summary.rename(columns={'施設名': '店名'}, inplace=True)
    expense_columns = ['賃料', '電気', '水道', '外部媒体手数料', 'その他経費']
    for col in expense_columns:
        if col not in pivot_summary.columns:
            pivot_summary[col] = 0

    pivot_summary['経費合計'] = pivot_summary[expense_columns].sum(axis=1)
    return pivot_summary

def main():
    print("--- 日別収支計算ツール (Colab版) ---")
    print("「売上」と「経費」のCSVファイルを2つ同時に選択してアップロードしてください。")

    # 1. ファイルアップロード
    uploaded = files.upload()

    if not uploaded:
        print("ファイルが選択されませんでした。")
        return

    sales_df = None
    expenses_df = None

    # 2. ファイル名による自動判別
    for filename, content in uploaded.items():
        if '売上' in filename:
            print(f"✅ 売上ファイルとして認識: {filename}")
            sales_df = pd.read_csv(io.BytesIO(content))
        elif '経費' in filename:
            print(f"✅ 経費ファイルとして認識: {filename}")
            expenses_df = pd.read_csv(io.BytesIO(content))

    # バリデーション
    if sales_df is None or expenses_df is None:
        print("\n❌ エラー: 「売上」と「経費」の両方のファイルが必要です。")
        print("ファイル名に「売上」または「経費」という文字が含まれているか確認してください。")
        return

    # 3. 集計処理
    today = date.today()
    print(f"\n集計を開始します（対象: {today} まで）...")

    sales_summary = process_sales(sales_df, today)
    expenses_summary = process_expenses(expenses_df, today)

    # 4. 結合（日別データ）
    if not sales_summary.empty and not expenses_summary.empty:
        merged_summary = pd.merge(
            sales_summary, expenses_summary, 
            on=['月次タグ', '日付', '店名'], how='outer'
        ).fillna(0)
    elif not sales_summary.empty:
        merged_summary = sales_summary
    elif not expenses_summary.empty:
        merged_summary = expenses_summary
    else:
        print("集計結果が空です。データを確認してください。")
        return

    # 5. カラム整理
    output_columns = [
        '月次タグ', '日付', '店名', '売上合計', 'ドロップイン売上', '設備予約売上',
        '店舗プラン売上', '外部媒体売上', 'その他売上', '経費合計', '賃料',
        '電気', '水道', '外部媒体手数料', 'その他経費'
    ]
    
    for col in output_columns:
        if col not in merged_summary.columns:
            merged_summary[col] = 0
    
    final_summary = merged_summary[output_columns].copy()
    final_summary = final_summary.sort_values(by=['月次タグ', '日付', '店名'])

    # 6. 保存とダウンロード
    final_summary.to_csv(OUTPUT_FILE_NAME, index=False, encoding='utf-8-sig')
    print(f"\n✨ 日別集計が完了しました！")
    print(f"データ件数: {len(final_summary)} 行")

    # 自動でダウンロードを開始
    files.download(OUTPUT_FILE_NAME)

if __name__ == '__main__':
    main()


--- 収支計算ツール (Colab版) ---
「売上」と「経費」のCSVファイルを2つ同時に選択してアップロードしてください。


Saving 新_直営売上管理_2026-01-19T17_28_14.537037011+09_00.csv to 新_直営売上管理_2026-01-19T17_28_14.537037011+09_00.csv
Saving 手打ち経費_2026-01-19T17_28_38.615911772+09_00.csv to 手打ち経費_2026-01-19T17_28_38.615911772+09_00.csv
✅ 売上ファイルとして認識: 新_直営売上管理_2026-01-19T17_28_14.537037011+09_00.csv
✅ 経費ファイルとして認識: 手打ち経費_2026-01-19T17_28_38.615911772+09_00.csv

集計を開始します（対象: 2026-01-19 まで）...

✨ 集計が完了しました！


/tmp/ipython-input-1799029105.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['月次'] = df['発生日'].dt.strftime('%Y-%m')
/tmp/ipython-input-1799029105.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['売上'] = clean_and_convert_to_numeric(df['売上'])
/tmp/ipython-input-1799029105.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⑥で生成されたデータの更新分を[連携シート](https://docs.google.com/spreadsheets/d/1EXbwGfBRXLl4AqHj1a4Jvkqvk5yRdsUp4z_1cGyx6gs/edit?usp=sharing)に貼り付けてください

In [ ]:
# @title ⑦日別進捗レポート生成ツール（5日刻み進捗分析対応版）
# ==========================================
# 1. システム準備
# ==========================================
print("--- [1/5] システム準備中 ---")
!pip install plotly -q

import pandas as pd
import json
import io
import unicodedata
import datetime
from google.colab import files
import sys
import re
import os

def clean_num(series):
    return pd.to_numeric(series.astype(str).str.replace('円', '').str.replace(',', ''), errors='coerce').fillna(0)

# ==========================================
# 2. ファイルアップロード
# ==========================================
print("\n" + "="*60)
print(" 【手順1】以下のCSVファイルをアップロードしてください。")
print(" 1. 全媒体日別統合データ_最終集計.csv（⑤の出力）")
print(" 2. 会議室シート_改 - グラフの順番.csv")
print(" 3. 会議室シート_改 - タグ設置リスト.csv")
print(" 4. daily_summary_final.csv（⑥の出力：連携シートデータ）")
print("="*60 + "\n")

uploaded = files.upload()

def find_file(keyword):
    for name in uploaded.keys():
        if keyword in unicodedata.normalize('NFC', name): return name
    return None

data_f = find_file("日別統合") or find_file("全媒体")
order_f = find_file("グラフの順番")
tags_f = find_file("タグ設置リスト")
store_f = find_file("daily_summary") or find_file("連携")

if not all([data_f, order_f, tags_f]):
    print("\n【エラー】必要なファイルが不足しています。")
    print(f"  日別統合データ: {'✅' if data_f else '❌'}")
    print(f"  グラフの順番: {'✅' if order_f else '❌'}")
    print(f"  タグ設置リスト: {'✅' if tags_f else '❌'}")
    print(f"  連携データ: {'✅' if store_f else '⚠️ (なくても続行可)'}")
    sys.exit()

# ==========================================
# 3. データの読み込みと日付自動検出
# ==========================================
print("\n--- [2/5] データ読み込み中 ---")

def load_csv(name):
    for enc in ['utf-8-sig', 'shift-jis', 'cp932', 'utf-8']:
        try: return pd.read_csv(io.BytesIO(uploaded[name]), encoding=enc)
        except: continue
    return pd.read_csv(io.BytesIO(uploaded[name]))

df_main = load_csv(data_f)
df_order = load_csv(order_f)
df_tags = load_csv(tags_f)
df_store_raw = load_csv(store_f) if store_f else pd.DataFrame()

# 日付列の処理
df_main['日付'] = pd.to_datetime(df_main['日付'], errors='coerce')
df_main = df_main.dropna(subset=['日付'])

# 最新日付を自動検出
cutoff_dt = df_main['日付'].max()
display_date = cutoff_dt.strftime('%Y年%m月%d日')
file_date_label = cutoff_dt.strftime('%Y%m%d')
current_day = cutoff_dt.day

print(f"✅ データの最新日付: {display_date}")

# 連携シートデータの処理
if not df_store_raw.empty and '日付' in df_store_raw.columns:
    df_store_raw['日付'] = pd.to_datetime(df_store_raw['日付'], errors='coerce')
    df_store_raw = df_store_raw.dropna(subset=['日付'])
    mapping = dict(zip(df_tags[df_tags['媒体名'] == 'いいアプリ']['媒体側施設名'], df_tags[df_tags['媒体名'] == 'いいアプリ']['統一施設名']))
    df_store_raw['統一店名'] = df_store_raw['店名'].map(mapping)

# 数値変換
df_main['売上'] = clean_num(df_main['売上'])
df_main['件数'] = clean_num(df_main['件数'])

if not df_store_raw.empty:
    df_store_raw['ドロップイン'] = clean_num(df_store_raw.get('ドロップイン売上', 0))
    df_store_raw['設備予約'] = clean_num(df_store_raw.get('設備予約売上', 0))
    df_store_raw['店舗プラン'] = clean_num(df_store_raw.get('店舗プラン売上', 0))
    df_store_raw['その他'] = clean_num(df_store_raw.get('その他売上', 0))

all_media = sorted(df_main['媒体名'].unique().tolist())
ordered_stores = df_order.iloc[:, 0].dropna().unique().tolist()

# ==========================================
# 4. 5日刻み進捗分析
# ==========================================
print("\n--- [3/5] 5日刻み進捗分析中 ---")

def get_milestone_day(day):
    """日付から該当するマイルストーン（5日刻み）を返す"""
    milestones = [5, 10, 15, 20, 25]
    for m in milestones:
        if day <= m:
            return m
    return 31  # 月末

def calc_cumulative_to_day(df, target_date, day_limit):
    """指定日までの累計を計算"""
    year_month = target_date.strftime('%Y-%m')
    start_date = pd.to_datetime(f"{year_month}-01")
    end_date = pd.to_datetime(f"{year_month}-{min(day_limit, 28):02d}")
    
    # 月末処理
    try:
        end_date = pd.to_datetime(f"{year_month}-{day_limit:02d}")
    except:
        # 月末を超える場合は月末まで
        import calendar
        last_day = calendar.monthrange(target_date.year, target_date.month)[1]
        end_date = pd.to_datetime(f"{year_month}-{last_day:02d}")
    
    mask = (df['日付'] >= start_date) & (df['日付'] <= end_date)
    return df[mask]['売上'].sum()

def calc_progress_data(df_media, df_store, store_name=None):
    """5日刻み進捗データを計算"""
    current_month = cutoff_dt.replace(day=1)
    prev_month = (current_month - pd.DateOffset(months=1))
    prev2_month = (current_month - pd.DateOffset(months=2))
    
    milestone = get_milestone_day(current_day)
    
    result = {
        'milestone': milestone,
        'current_day': current_day,
        'categories': {}
    }
    
    # 媒体データの集計
    df_filtered = df_media if store_name is None else df_media[df_media['統一店名'] == store_name]
    
    for media in all_media:
        df_m = df_filtered[df_filtered['媒体名'] == media]
        
        # 当月・前月・前々月の累計（同日まで）
        curr_total = calc_cumulative_to_day(df_m, cutoff_dt, current_day)
        
        # 前月の同日までの累計
        prev_date = prev_month.replace(day=min(current_day, 28))
        try:
            prev_date = prev_month.replace(day=current_day)
        except:
            import calendar
            last_day = calendar.monthrange(prev_month.year, prev_month.month)[1]
            prev_date = prev_month.replace(day=min(current_day, last_day))
        prev_total = calc_cumulative_to_day(df_m, prev_date, current_day)
        
        # 前々月の同日までの累計
        prev2_date = prev2_month.replace(day=min(current_day, 28))
        try:
            prev2_date = prev2_month.replace(day=current_day)
        except:
            import calendar
            last_day = calendar.monthrange(prev2_month.year, prev2_month.month)[1]
            prev2_date = prev2_month.replace(day=min(current_day, last_day))
        prev2_total = calc_cumulative_to_day(df_m, prev2_date, current_day)
        
        result['categories'][media] = {
            'current': int(curr_total),
            'prev': int(prev_total),
            'prev2': int(prev2_total),
            'vs_prev': round(curr_total / prev_total * 100, 1) if prev_total > 0 else 0,
            'vs_prev2': round(curr_total / prev2_total * 100, 1) if prev2_total > 0 else 0
        }
    
    # 連携シートデータ（ドロップイン・設備予約）
    if not df_store.empty:
        df_s = df_store if store_name is None else df_store[df_store['統一店名'] == store_name]
        
        for cat in ['ドロップイン', '設備予約']:
            col = cat
            if col in df_s.columns:
                df_cat = df_s.copy()
                df_cat['売上'] = df_cat[col]
                
                curr_total = calc_cumulative_to_day(df_cat, cutoff_dt, current_day)
                prev_total = calc_cumulative_to_day(df_cat, prev_date, current_day)
                prev2_total = calc_cumulative_to_day(df_cat, prev2_date, current_day)
                
                result['categories'][cat] = {
                    'current': int(curr_total),
                    'prev': int(prev_total),
                    'prev2': int(prev2_total),
                    'vs_prev': round(curr_total / prev_total * 100, 1) if prev_total > 0 else 0,
                    'vs_prev2': round(curr_total / prev2_total * 100, 1) if prev2_total > 0 else 0
                }
    
    return result

# 全社進捗データ
progress_all = calc_progress_data(df_main, df_store_raw)

# 店舗別進捗データ
progress_stores = {}
for store in ordered_stores:
    progress_stores[store] = calc_progress_data(df_main, df_store_raw, store)

# ==========================================
# 5. 13ヶ月月次推移データ（既存機能）
# ==========================================
print("\n--- [4/5] 月次推移データ構築中 ---")

months_13_dt = [(cutoff_dt.replace(day=1) - pd.DateOffset(months=i)) for i in range(12, -1, -1)]
month_labels = [d.strftime('%Y年%m月') for d in months_13_dt]

def calc_pct(curr, prev):
    if prev == 0: return "-" if curr == 0 else "新規"
    return f"{int((curr / prev) * 100)}%"

def get_monthly_data(s_name=None, f_name=None, room_only_global=False):
    res_list = []
    for d, label in zip(months_13_dt, month_labels):
        row = {"month": label}
        month_start = d
        month_end = (d + pd.DateOffset(months=1)) - pd.Timedelta(days=1)
        
        if s_name and not f_name:  # 店舗全体
            if not df_store_raw.empty:
                s_base = df_store_raw[(df_store_raw['統一店名']==s_name) & (df_store_raw['日付']>=month_start) & (df_store_raw['日付']<=month_end)]
                row.update({"ドロップイン": int(s_base['ドロップイン'].sum()), "設備予約": int(s_base['設備予約'].sum()), "店舗プラン": int(s_base['店舗プラン'].sum()), "その他": int(s_base['その他'].sum())})
            else:
                row.update({"ドロップイン": 0, "設備予約": 0, "店舗プラン": 0, "その他": 0})
            
            s_room = df_main[(df_main['統一店名']==s_name) & (df_main['日付']>=month_start) & (df_main['日付']<=month_end)]
            for m in all_media: row[m] = int(s_room[s_room['媒体名']==m]['売上'].sum())
            row["total_sales"] = row["ドロップイン"] + row["設備予約"] + row["店舗プラン"] + row["その他"] + sum([row[m] for m in all_media])
        
        elif s_name and f_name:  # 設備詳細
            f_rows = df_main[(df_main['統一店名']==s_name) & (df_main['統一設備名']==f_name) & (df_main['日付']>=month_start) & (df_main['日付']<=month_end)]
            row.update({"total_sales": int(f_rows['売上'].sum()), "total_counts": int(f_rows['件数'].sum())})
            for m in all_media:
                m_r = f_rows[f_rows['媒体名']==m]
                row[f"s_{m}"] = int(m_r['売上'].sum())
                row[f"c_{m}"] = int(m_r['件数'].sum())
        
        elif room_only_global:  # 全社会議室のみ
            s_room = df_main[(df_main['日付']>=month_start) & (df_main['日付']<=month_end)]
            row["total_sales"] = int(s_room['売上'].sum())
            for m in all_media: row[m] = int(s_room[s_room['媒体名']==m]['売上'].sum())
        
        else:  # 全社総合
            if not df_store_raw.empty:
                s_base = df_store_raw[(df_store_raw['日付']>=month_start) & (df_store_raw['日付']<=month_end)]
                row.update({"ドロップイン": int(s_base['ドロップイン'].sum()), "設備予約": int(s_base['設備予約'].sum()), "店舗プラン": int(s_base['店舗プラン'].sum()), "その他": int(s_base['その他'].sum())})
            else:
                row.update({"ドロップイン": 0, "設備予約": 0, "店舗プラン": 0, "その他": 0})
            
            s_room = df_main[(df_main['日付']>=month_start) & (df_main['日付']<=month_end)]
            for m in all_media: row[m] = int(s_room[s_room['媒体名']==m]['売上'].sum())
            row["total_sales"] = row["ドロップイン"] + row["設備予約"] + row["店舗プラン"] + row["その他"] + sum([row[m] for m in all_media])
        
        res_list.append(row)
    
    for i in range(len(res_list)):
        curr, prev = res_list[i], (res_list[i-1] if i > 0 else None)
        res_list[i]["total_sales_mom"] = calc_pct(curr["total_sales"], prev["total_sales"] if prev else 0)
    
    return res_list

# JSON構築
js_master = {
    "g_all": get_monthly_data(), 
    "g_room": get_monthly_data(room_only_global=True),
    "progress_all": progress_all,
    "stores": []
}

for idx, sname in enumerate(ordered_stores):
    s_main = df_main[df_main['統一店名'] == sname]
    sorted_facs = s_main.groupby('統一設備名')['売上'].sum().sort_values(ascending=False).index.tolist() if not s_main.empty else []
    fac_data = [{"name": f, "data": get_monthly_data(sname, f)} for f in sorted_facs]
    js_master["stores"].append({
        "id": f"st_{idx}", 
        "name": sname, 
        "summary": get_monthly_data(sname), 
        "progress": progress_stores.get(sname, {}),
        "facilities": fac_data
    })

# ==========================================
# 6. HTML生成（5日刻み進捗分析付き）
# ==========================================
print("\n--- [5/5] HTMLを生成中 ---")

json_data_str = json.dumps(js_master, ensure_ascii=False)
media_list_str = json.dumps(all_media, ensure_ascii=False)

current_month_label = cutoff_dt.strftime('%Y年%m月')
prev_month_label = (cutoff_dt - pd.DateOffset(months=1)).strftime('%Y年%m月')
prev2_month_label = (cutoff_dt - pd.DateOffset(months=2)).strftime('%Y年%m月')

html_template = f"""
<!DOCTYPE html><html><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1">
<title>日別進捗ダッシュボード - {display_date}時点</title><script src="https://cdn.plot.ly/plotly-2.24.1.min.js"></script>
<style>
:root {{ --primary: #22923f; --primary-light: #e8f8ed; --text: #2d3436; --bg: #f8f9fa; --border: #dfe6e9; --up: #e74c3c; --down: #2980b9; }}
body {{ font-family: sans-serif; margin: 0; display: flex; background: var(--bg); color: var(--text); }}
nav {{ width: 260px; background: #2f3640; height: 100vh; position: sticky; top: 0; overflow-y: auto; color: white; flex-shrink: 0; z-index: 1000; }}
nav h3 {{ padding: 20px; background: #222f3e; margin: 0; font-size: 1rem; }}
nav ul {{ list-style: none; padding: 0; }}
nav li a {{ display: block; padding: 12px 20px; color: #ced6e0; text-decoration: none; border-bottom: 1px solid #34495e; font-size: 0.82rem; }}
nav li a:hover {{ background: var(--primary); color: white; }}

.title-bar {{ position: sticky; top: 0; background: white; padding: 10px 25px; border-bottom: 3px solid var(--primary); box-shadow: 0 2px 5px rgba(0,0,0,0.08); z-index: 900; display: flex; justify-content: space-between; align-items: center; }}
.title-bar h2 {{ margin: 0; font-size: 1.2rem; color: var(--primary); }}

main {{ flex: 1; padding: 0; min-width: 0; }}
.section {{ padding: 10px 25px 80px; scroll-margin-top: 10px; }}
.card {{ background: white; border-radius: 12px; box-shadow: 0 4px 15px rgba(0,0,0,0.05); margin-bottom: 35px; border: 1px solid var(--border); overflow: hidden; }}
.card-h {{ background: #f8f9fa; padding: 12px 20px; font-weight: bold; border-bottom: 1px solid var(--border); }}
.card-b {{ padding: 20px; }}
.chart-box {{ height: 380px; width: 100%; margin-bottom: 15px; }}
.table-c {{ overflow-x: auto; border-radius: 8px; border: 1px solid var(--border); margin-top: 15px; }}
table {{ width: 100%; border-collapse: collapse; font-size: 0.78rem; background: white; }}
th, td {{ border: 1px solid #eee; padding: 8px; text-align: right; }}
th {{ background: #f1f2f6; text-align: center; white-space: nowrap; }}
td:first-child {{ text-align: center; font-weight: bold; background: #fafafa; position: sticky; left: 0; }}
.mom {{ font-weight: bold; font-size: 0.75rem; }}
.mom.up {{ color: var(--up); }}
.mom.down {{ color: var(--down); }}
.tot {{ background: var(--primary-light); font-weight: bold; }}

/* 5日刻み進捗スタイル */
.progress-card {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 12px; padding: 20px; margin-bottom: 20px; }}
.progress-card h3 {{ margin: 0 0 15px 0; font-size: 1.1rem; }}
.progress-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(180px, 1fr)); gap: 15px; }}
.progress-item {{ background: rgba(255,255,255,0.15); border-radius: 8px; padding: 15px; }}
.progress-item .label {{ font-size: 0.85rem; opacity: 0.9; margin-bottom: 5px; }}
.progress-item .value {{ font-size: 1.5rem; font-weight: bold; }}
.progress-item .compare {{ font-size: 0.75rem; margin-top: 5px; }}
.progress-item .compare span {{ padding: 2px 6px; border-radius: 4px; margin-right: 5px; }}
.progress-item .compare .up {{ background: rgba(231,76,60,0.3); }}
.progress-item .compare .down {{ background: rgba(41,128,185,0.3); }}

.menu-btn {{ display: none; position: fixed; bottom: 20px; right: 20px; background: var(--primary); color: white; border: none; border-radius: 50%; width: 60px; height: 60px; font-size: 24px; z-index: 2000; box-shadow: 0 5px 15px rgba(0,0,0,0.2); }}
@media (max-width: 1024px) {{ nav {{ position: fixed; left: -260px; transition: 0.3s; }} nav.open {{ left: 0; }} .menu-btn {{ display: block; }} }}
</style></head><body>
<button class="menu-btn" onclick="document.getElementById('sidebar').classList.toggle('open')">☰</button>
<nav id="sidebar"><h3>📊 進捗ダッシュボード</h3><ul id="nav-ul"></ul></nav>
<main id="main-c"></main>
<script>
const data = {json_data_str};
const media = {media_list_str};
const content = document.getElementById('main-c');
const nav = document.getElementById('nav-ul');
const currentDay = {current_day};
const milestone = data.progress_all.milestone;

let nHtml = '<li><a href="#g-all" onclick="closeMenu()">📈 全店舗合計</a></li>';
nHtml += '<li><a href="#g-room" onclick="closeMenu()">💼 会議室のみ</a></li>';
data.stores.forEach(s => {{ nHtml += `<li><a href="#${{s.id}}" onclick="closeMenu()">${{s.name}}</a></li>`; }});
nav.innerHTML = nHtml;

function closeMenu() {{ document.getElementById('sidebar').classList.remove('open'); }}
function getMomClass(m) {{ if(!m || m==="-" || m==="新規") return ""; const v = parseInt(m.replace('%','')); return v > 100 ? "up" : (v < 100 ? "down" : ""); }}
function formatNum(n) {{ return n.toLocaleString(); }}
function getPctClass(v) {{ return v > 100 ? "up" : (v < 100 ? "down" : ""); }}

function getProgressHtml(prog) {{
    if(!prog || !prog.categories) return '';
    let h = `<div class="progress-card"><h3>📅 ${{currentDay}}日時点の進捗（${{milestone}}日マイルストーン基準）</h3><div class="progress-grid">`;
    
    const cats = ['ドロップイン', '設備予約', ...media];
    cats.forEach(cat => {{
        const d = prog.categories[cat];
        if(!d) return;
        const prevClass = getPctClass(d.vs_prev);
        const prev2Class = getPctClass(d.vs_prev2);
        h += `<div class="progress-item">
            <div class="label">${{cat}}</div>
            <div class="value">¥${{formatNum(d.current)}}</div>
            <div class="compare">
                <span class="${{prevClass}}">前月比${{d.vs_prev}}%</span>
                <span class="${{prev2Class}}">前々月比${{d.vs_prev2}}%</span>
            </div>
        </div>`;
    }});
    
    return h + '</div></div>';
}}

function getStoreTbl(rows, cats) {{
    let h = '<div class="table-c"><table><thead><tr><th>月次</th><th>合計売上</th><th>前月比</th>';
    cats.forEach(c => h += `<th>${{c}}</th>`);
    h += '</tr></thead><tbody>';
    rows.forEach(r => {{
        h += `<tr><td>${{r.month}}</td><td class="tot">${{r.total_sales.toLocaleString()}}</td><td class="mom ${{getMomClass(r.total_sales_mom)}}">${{r.total_sales_mom}}</td>`;
        cats.forEach(c => h += `<td>${{(r[c]||0).toLocaleString()}}</td>`);
        h += '</tr>';
    }});
    return h + '</tbody></table></div>';
}}

function getFacTbl(rows) {{
    let h = '<div class="table-c"><table><thead><tr><th rowspan="2">月次</th><th colspan="3" style="background:#e8f8ed">施設全体</th>';
    media.forEach(m => h += `<th colspan="2">${{m}}</th>`);
    h += '</tr><tr><th style="background:#f1f2f6">合計売上</th><th style="background:#f1f2f6">前月比</th><th style="background:#f1f2f6">合計件数</th>';
    media.forEach(m => h += `<th>売上</th><th>件数</th>`);
    h += '</tr></thead><tbody>';
    rows.forEach(r => {{
        h += `<tr><td>${{r.month}}</td><td class="tot">${{r.total_sales.toLocaleString()}}</td><td class="mom ${{getMomClass(r.total_sales_mom)}}">${{r.total_sales_mom}}</td><td>${{r.total_counts.toLocaleString()}}</td>`;
        media.forEach(m => {{ h += `<td>${{(r['s_'+m]||0).toLocaleString()}}</td><td>${{(r['c_'+m]||0).toLocaleString()}}</td>`; }});
        h += '</tr>';
    }});
    return h + '</tbody></table></div>';
}}

let sHtml = `<section id="g-all" class="section">
    <div class="title-bar"><h2>📊 全店舗合計</h2><span>データ時点: {display_date}</span></div>
    ${{getProgressHtml(data.progress_all)}}
    <div class="card"><div class="card-h">月次推移（13ヶ月）</div><div class="card-b"><div id="c-gall" class="chart-box"></div><div id="t-gall"></div></div></div>
</section>
<section id="g-room" class="section">
    <div class="title-bar"><h2>💼 会議室売上のみ</h2></div>
    <div class="card"><div class="card-h">月次推移（13ヶ月）</div><div class="card-b"><div id="c-groom" class="chart-box"></div><div id="t-groom"></div></div></div>
</section>`;

data.stores.forEach(s => {{
    sHtml += `<section id="${{s.id}}" class="section">
        <div class="title-bar"><h2>🏠 ${{s.name}}</h2><span>データ時点: {display_date}</span></div>
        ${{getProgressHtml(s.progress)}}
        <div class="card"><div class="card-h">店舗全体実績</div><div class="card-b"><div id="c-s-${{s.id}}" class="chart-box"></div><div id="t-s-${{s.id}}"></div></div></div>`;
    s.facilities.forEach((f, i) => {{
        sHtml += `<div class="card"><div class="card-h">【設備】${{f.name}}</div><div class="card-b"><div style="display:flex; flex-wrap:wrap; gap:15px;"><div id="c-fs-${{s.id}}-${{i}}" class="chart-box" style="flex:1; min-width:320px;"></div><div id="c-fc-${{s.id}}-${{i}}" class="chart-box" style="flex:1; min-width:320px;"></div></div><div id="t-f-${{s.id}}-${{i}}"></div></div></div>`;
    }});
    sHtml += '</section>';
}});
content.innerHTML = sHtml;

const rendered = new Set();
function render(id) {{
    if(rendered.has(id)) return;
    const cfg = {{ responsive:true, displayModeBar:false }};
    const lay = {{ barmode:'stack', margin:{{t:40,b:40,l:60,r:15}}, font:{{size:11}}, legend:{{orientation:'h',y:-0.15}}, colorway:['#22923f','#3498db','#9b59b6','#f1c40f','#e67e22','#e74c3c','#1abc9c','#34495e'] }};
    if(id==='g-all'){{
        const cats = ["ドロップイン","設備予約","店舗プラン","その他",...media];
        const trs = cats.map(c=>({{x:data.g_all.map(r=>r.month),y:data.g_all.map(r=>r[c]),name:c,type:'bar'}})).filter(t=>t.y.some(v=>v>0));
        Plotly.newPlot('c-gall',trs,lay,cfg); document.getElementById('t-gall').innerHTML = getStoreTbl(data.g_all,cats);
    }} else if(id==='g-room'){{
        const trs = media.map(c=>({{x:data.g_room.map(r=>r.month),y:data.g_room.map(r=>r[c]),name:c,type:'bar'}})).filter(t=>t.y.some(v=>v>0));
        Plotly.newPlot('c-groom',trs,lay,cfg); document.getElementById('t-groom').innerHTML = getStoreTbl(data.g_room,media);
    }} else {{
        const s = data.stores.find(x => x.id === id); if(!s) return;
        const cats = ["ドロップイン","設備予約","店舗プラン","その他",...media];
        const trs = cats.map(c=>({{x:s.summary.map(r=>r.month),y:s.summary.map(r=>r[c]),name:c,type:'bar'}})).filter(t=>t.y.some(v=>v>0));
        Plotly.newPlot('c-s-'+id,trs,lay,cfg); document.getElementById('t-s-'+id).innerHTML = getStoreTbl(s.summary,cats);
        s.facilities.forEach((f,i)=>{{
            const ftrs = media.map(m=>({{x:f.data.map(r=>r.month),y:f.data.map(r=>r["s_"+m]),name:m,type:'bar'}})).filter(t=>t.y.some(v=>v>0));
            const fctrs = media.map(m=>({{x:f.data.map(r=>r.month),y:f.data.map(r=>r["c_"+m]),name:m,type:'bar'}})).filter(t=>t.y.some(v=>v>0));
            Plotly.newPlot(`c-fs-${{id}}-${{i}}`,ftrs,{{...lay, title:'売上推移'}},cfg);
            Plotly.newPlot(`c-fc-${{id}}-${{i}}`,fctrs,{{...lay, title:'件数推移'}},cfg);
            document.getElementById(`t-f-${{id}}-${{i}}`).innerHTML = getFacTbl(f.data);
        }});
    }}
    rendered.add(id);
}}
const obs = new IntersectionObserver((es)=>{{ es.forEach(e=>{{ if(e.isIntersecting) render(e.target.id); }}); }}, {{threshold:0.05}});
document.querySelectorAll('.section').forEach(s=>obs.observe(s));
</script></body></html>
"""

# ファイル書き出し
filename = f"日別進捗ダッシュボード_{file_date_label}.html"
with open(filename, "w", encoding="utf-8") as f:
    f.write(html_template)
print(f"\n--- 完了！ ---")
print(f"✅ {filename} をダウンロードします")
files.download(filename)


--- [1/5] システム準備中 ---

 【手順1】以下の4つのCSVファイルをアップロードしてください。
 1. 会議室シート_改 - 全体元データ.csv
 2. 会議室シート_改 - グラフの順番.csv
 3. 会議室シート_改 - タグ設置リスト.csv
 4. 新_直営店舗売上データ - 連携.csv



Saving 新_直営店舗売上データ - 連携 (1).csv to 新_直営店舗売上データ - 連携 (1).csv
Saving 会議室シート_改 - タグ設置リスト (5).csv to 会議室シート_改 - タグ設置リスト (5).csv
Saving 会議室シート_改 - グラフの順番 (2).csv to 会議室シート_改 - グラフの順番 (2).csv
Saving 会議室シート_改 - 全体元データ (2).csv to 会議室シート_改 - 全体元データ (2).csv

【手順2】何日までのデータですか？（例: 2026/01/20）: 2026/01/19

--- [3/5] 2026年01月19日を基準に13ヶ月分を集計中 ---

--- [4/5] ダッシュボード構造を構築中 ---

--- [5/5] HTMLを生成中 ---

--- [5/5] 完了！ ---


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>